In [0]:
import bodo
bodo.__version__

Out[1]: '2021.8'

In [0]:
import ipyparallel as ipp
ipp.__version__

Out[1]: '7.1.0'

In [0]:
import os
import sys
import json
from subprocess import run, STDOUT

os.environ['OMPI_ALLOW_RUN_AS_ROOT'] = '1'
os.environ['OMPI_ALLOW_RUN_AS_ROOT_CONFIRM'] = '1'

In [0]:
# command to stop cluster
# c.stop_cluster_sync()

Stopping controller
INFO:ipyparallel.cluster.cluster.1634245268-ddiv:Stopping controller
Stopping engine(s): 1634245368
INFO:ipyparallel.cluster.cluster.1634245268-ddiv:Stopping engine(s): 1634245368

In [0]:
c = ipp.Cluster(profile="mpi",
                engine_launcher_class='MPI',
                n=n_cores,  # Number of engines, you can change this
                controller_ip='*',
                controller_args=["--nodb"],
                controller_location=spark.conf.get('spark.driver.host')
               )
c.engine_launcher_class.mpi_args = ["-machinefile", "/home/ubuntu/.ssh/hostfile"]
c.start_controller_sync()

Using existing profile dir: '/root/.ipython/profile_mpi'

In [0]:
# Get connection info
connection_info = await c.controller.get_connection_info()
engine_info = connection_info['engine']

In [0]:
os.environ['CONNECTION_INFO'] = json.dumps(engine_info)
os.environ['connection_file'] = os.path.join(
        c.profile_dir,
        'security',
        f'ipcontroller-{c.cluster_id}-engine.json',
    )

In [0]:
# Get connection info
connection_info = await c.controller.get_connection_info()
engine_info = connection_info['engine']



def send_connection_info(connection_info, connection_file):
    env = os.environ.copy()
    env["CONNECTION_INFO"] = json.dumps(connection_info)
    cmd =     [
        'mpiexec',
        '-ppn',
        '1',
        '-machinefile',
        "/home/ubuntu/.ssh/hostfile",
        'sh',
        '-c',
        f'echo $CONNECTION_INFO > "{connection_file}"'

    ]
    p = run(cmd, capture_output=True, text=True, input=None, env=env)
    if p.returncode:
        print(p.stderr, file=sys.stderr)
        p.check_returncode()
    return p

send_connection_info(
    engine_info,
    os.path.join(
        c.profile_dir,
        'security',
        f'ipcontroller-{c.cluster_id}-engine.json',
    ),
)

Out[6]: CompletedProcess(args=['mpiexec', '-ppn', '1', '-machinefile', '/home/ubuntu/.ssh/hostfile', 'sh', '-c', 'echo $CONNECTION_INFO > "/root/.ipython/profile_mpi/security/ipcontroller-1634246350-5u77-engine.json"'], returncode=0, stdout='', stderr='')

In [0]:
c.start_engines_sync()
rc = c.connect_client_sync()
rc.wait_for_engines(n=c.n)
view = rc[:]
view.activate()
view.block = True

Starting 12 engines with <class 'ipyparallel.cluster.launcher.MPIEngineSetLauncher'>
INFO:ipyparallel.cluster.cluster.1634246350-5u77:Starting 12 engines with <class 'ipyparallel.cluster.launcher.MPIEngineSetLauncher'>
 0%| | 0/12 [00:00<?, ?engine/s] 8%|▊ | 1/12 [00:04<00:54, 4.93s/engine]100%|██████████| 12/12 [00:04<00:00, 2.43engine/s]

In [0]:
%%px
from mpi4py import MPI
comm = MPI.COMM_WORLD
print(f"Hello World from rank {comm.Get_rank()} on host {MPI.Get_processor_name()}. total ranks={comm.Get_size()}")

[stdout:1] 
Hello World from rank 1 on host 1014-162708-fujf0q9l-10-44-233-230. total ranks=12
[stdout:2] 
Hello World from rank 2 on host 1014-162708-fujf0q9l-10-44-227-112. total ranks=12
[stdout:0] 
Hello World from rank 0 on host 1014-162708-fujf0q9l-10-44-235-163. total ranks=12
[stdout:3] 
Hello World from rank 3 on host 1014-162708-fujf0q9l-10-44-235-163. total ranks=12
[stdout:11] 
Hello World from rank 11 on host 1014-162708-fujf0q9l-10-44-227-112. total ranks=12
[stdout:4] 
Hello World from rank 4 on host 1014-162708-fujf0q9l-10-44-233-230. total ranks=12
[stdout:6] 
Hello World from rank 6 on host 1014-162708-fujf0q9l-10-44-235-163. total ranks=12
[stdout:5] 
Hello World from rank 5 on host 1014-162708-fujf0q9l-10-44-227-112. total ranks=12
[stdout:7] 
Hello World from rank 7 on host 1014-162708-fujf0q9l-10-44-233-230. total ranks=12
[stdout:8] 
Hello World from rank 8 on host 1014-162708-fujf0q9l-10-44-227-112. total ranks=12
[stdout:10] 
Hello World from rank 10 on host 1014-162708-fujf0q9l-10-44-233-230. total ranks=12
[stdout:9] 
Hello World from rank 9 on host 1014-162708-fujf0q9l-10-44-235-163. total ranks=12